In [3]:
#from elasticsearch import Elasticsearch
from pyelasticsearch import ElasticSearch #see if this is better than 'official API' - the docs look better to me
import json
from jsmin import jsmin

In [5]:
%install_ext https://raw.githubusercontent.com/szeitlin/watermark/master/watermark.py

Installed watermark.py. To use it, type:
  %load_ext watermark


In [6]:
%load_ext watermark

In [7]:
%watermark -a "Samantha Zeitlin" -d -u -p elasticsearch,pyelasticsearch,jsmin,python

Samantha Zeitlin 
Last updated: 04/09/2015 

elasticsearch 1.4.0
pyelasticsearch 1.2.1
jsmin 2.1.1
python 2.7.9


In [16]:
#try doing two files together (if this works, put target files in folder and iterate through that)
with open("elk_test/import-one-year.ipynb", 'r') as test:
    minified = jsmin(test.read())
with open("elk_test/2013_2014_comparison_prep.ipynb", 'r') as test2:
    mini2 = jsmin(test2.read())
    with open('pyminified_testfile.json', 'a+') as pyminified:
        pyminified.write(prefix)
        pyminified.write(minified)
        pyminified.write(prefix)
        pyminified.write(mini2)

In [11]:
es = ElasticSearch('http://localhost:9200/')
#es.indices.create(index='test-index')

In [35]:
es.bulk([es.index_op(minified), es.index_op(mini2)], doc_type='notebook', index='test2')

BulkError: 2 of 2 bulk actions failed.

In [66]:
!ls

Dockerfile                                 another_query.txt                          first_try_mapping.json                     minified_testfile.json                     testcreate.json
LICENSE                                    draft_bulk_import_multiple_notebooks.ipynb import_statements_query.txt                pyminified_testfile.json                   testnb.json
README.md                                  elasticsearch.yml                          match_query.txt                            test2.json                                 testprefix.json
action.json                                elk_test                                   minified.json                              testbulk.json                              testquery.txt


In [28]:
with open('import_statements_query.txt', 'r') as query:
    q=query.read()
    es.search(q, index='test2')

In [21]:
%%bash
curl -s -XPOST localhost:9200/_bulk --data-binary @test2.json


{"took":251,"errors":false,"items":[{"index":{"_index":"test","_type":"type1","_id":"2","_version":1,"status":201}}]}

In [ ]:
#looks like kibana barely works at all with only 1 document, suddenly with 2 it starts showing more functionality. 